In [1]:
import pandas as pd

summary = pd.read_csv('news_summary.csv',
                      encoding='iso-8859-1')
raw = pd.read_csv('news_summary_more.csv',
                  encoding='iso-8859-1')

In [2]:
r = raw.copy()
s = summary.copy()

# To increase the intake of possible text values to build a reliable model
s['text'] = s['author'].str.cat(s['date'
        ].str.cat(s['read_more'].str.cat(s['text'
        ].str.cat(s['ctext'], sep=' '), sep=' '), sep=' '), sep=' ')

pre = pd.DataFrame()
pd.set_option('display.max_colwidth', None)
pre['text'] = pd.concat([r['text'], s['text']], ignore_index=True)
pre['summary'] = pd.concat([r['headlines'], s['headlines']],
                           ignore_index=True)

In [3]:
duplicate_rows = pre[pre.duplicated()]
print("Duplicate rows count: ", duplicate_rows.shape)

Duplicate rows count:  (22, 2)


In [4]:
pre.drop_duplicates()

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems En

# Old code. Dont use 

from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

for index, row in pre.iterrows():
    if index == 5:
        break

    text = row['text']
    
    # Encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)

    # Generate summary with the model
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=70, early_stopping=True)

    # Decode the tensor to human-readable text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"Orginal text ; {text}")
    print(f"\nGenerated summary ; {summary}")
    print(f"\nTest summary: {row['summary']}")
    print("\n")


In [ ]:
# from transformers import pipeline

# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# for index, row in pre.iterrows():
#     if index == 200:
#         break
#     print("\nNews: {}".format(row['text']))
# #     print("\nOriginal Summary: {}".format(row['summary']))
#     text = row['text']
#     output = summarizer(text, max_length=45, min_length=0, do_sample=False)
#     print(output[0]['summary_text'])


In [6]:
from transformers import pipeline
from rouge_score import rouge_scorer

# Initialize the summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

f1_scores_rouge1 = []
f1_scores_rouge2 = []
f1_scores_rougeL = []

for index, row in pre.iterrows():
    if index == 200:
        break
    original_text = row['text']
    reference_summary = row['summary']

    # Generate summary
    generated_summary = summarizer(original_text, max_length=45, min_length=0, do_sample=False)[0]['summary_text']

    # Print original and generated summaries
    print(f"\nReview: {original_text}")
    print(f"\nOriginal Summary: {reference_summary}")
    print(f"\nPredicted Summary: {generated_summary}")

    # Compute ROUGE scores
    scores = scorer.score(reference_summary, generated_summary)
#     print("\nROUGE scores:", scores)
#     print("\n")

    f1_scores_rouge1.append(scores['rouge1'].fmeasure)
    f1_scores_rouge2.append(scores['rouge2'].fmeasure)
    f1_scores_rougeL.append(scores['rougeL'].fmeasure)



Review: Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.

Original Summary: upGrad learner switches to career in ML & Al with 90% salary hike

Predicted Summary: Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work

Review: Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, 


Review: Former stripper and regional sales director of a pharmaceutical company, Sunrise Lee, gave a doctor a lap dance in a nightclub to persuade him to prescribe an addictive fentanyl spray in 2012, the company's sales representative told a US court. She said she saw Lee "sitting on [doctor's] lap, kind of bouncing around." Lee has been accused of bribing doctors.

Original Summary: Pharma exec gave doctor a lap dance to sell medicine in US: Witness

Predicted Summary:  Sunrise Lee is a former stripper and regional sales director of a pharmaceutical company. She gave a doctor a lap dance in a nightclub to persuade him to prescribe an addictive fentanyl spray.

Review: Reliance Industries' Chairman Mukesh Ambani's daughter Isha Ambani, who got married last month, said she only cried at her 'bidaai' because she felt peer pressure as everyone was crying, especially her parents. "I was emotional too but everyone around me would cry all the time," she added. "It was a very emotional affa


Review: A London zoo is offering people to name a cockroach after their exes on Valentine's Day for ÃÂ£1.50 (nearly Ã¢ÂÂ¹140). "For those that don't quite require revenge, there's another way to make you feel better about getting back at your ex," the zoo said in a statement. The names will appear on zoo's 'roach board' on February 14. 

Original Summary: UK zoo offers people to name cockroach after their ex on Valentine's

Predicted Summary: A London zoo is offering people to name a cockroach after their exes on Valentine's Day. The names will appear on zoo's 'roach board' on February 14.

Review: Stand-in captain Rohit Sharma has become the 14th Indian cricketer to play 200 ODIs, achieving the feat after taking the field against New Zealand in fourth ODI at Hamilton on Thursday. The 31-year-old had made his ODI debut on June 23, 2007, against Ireland in Belfast. Rohit scored 7,799 runs in his first 199 ODIs at an average of 48.14.

Original Summary: Rohit Sharma becomes 14th India


Review: Gabrielle Reilly, a 22-year-old US woman, created earrings for her Apple AirPods and put it up for sale online for $20 (about Ã¢ÂÂ¹1,500). "I absolutely refuse to lose them (AirPods)...So I made earrings," she explained. The earrings, which took her over an hour to make, debuted in a video on Twitter that has since garnered over three million views.

Original Summary: Woman turns Apple AirPods into earrings to avoid losing them

Predicted Summary: Gabrielle Reilly, a 22-year-old US woman, created earrings for her Apple AirPods. The earrings, which took her over an hour to make, debuted in a video on

Review: Ex-ICICI Bank CEO Chanda Kochhar said she's "utterly disappointed, hurt and shocked" by the bank's decision to terminate her. Kochhar said she served ICICI for 34 years with utmost honesty and integrity and that ultimately truth will prevail. Adding that she hasn't been given a copy of the independent probe report, Kochhar said none of ICICI's credit decisions are unilate

Your max_length is set to 45, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


...


Review: Congress MP Shashi Tharoor has shared an image of Uttar Pradesh CM Yogi Adityanath taking a holy dip at Kumbh Mela along with his Cabinet ministers. "You want to keep the Ganga clean and also wash your sins in it. Everyone is naked in this Sangam. Jai Ganga Maiya ki," Tharoor's caption on the picture read.

Original Summary: You want Ganga clean & also wash sins in it: Tharoor on UP CM's pic

Predicted Summary: Congress MP Shashi Tharoor shared an image of Uttar Pradesh CM Yogi Adityanath taking a holy dip at Kumbh Mela. "You want to keep the Ganga clean and also

Review: Three women were arrested on Tuesday in connection with the poisoning of prasad at a temple in Chikkaballapur district, Karnataka. Two women had died while several other people were hospitalised with complaints of vomiting and loose motions. The police had earlier said the temple management was not responsible as the women had brought the prasad themselves and were distributing it. 

Original Summary: 3 women


Review: Developer platform 'Crio.Do' has raised Ã¢ÂÂ¹7 crore ($1 million) from investors including Flipkart Co-founder Binny Bansal and B2B online marketplace Udaan's Co-founders Amod Malviya, Vaibhav Gupta and Sujeet Kumar. Mekin Maheshwari, Udhyam Learning Foundation's founder, also participated in the funding round. Founded in 2018, the Bengaluru-based startup offers a learning environment for developers.

Original Summary: Developer platform raises Ã¢ÂÂ¹7 crore from Binny Bansal, others

Predicted Summary: Developer platform 'Crio.Do' has raised Ã¢ÂÂ¹7 crore ($1 million) from investors. Flipkart Co-founder Binny Bansal and B2B

Review: NASA's Curiosity rover has shared its last selfie clicked on Mars' Vera Rubin Ridge, which was its home for over a year. A series of 57 pictures were stitched together to create the selfie. Curiosity had drilled its 19th sample at 'Rock Hall' on the ridge on December 15, 2018, which is also visible in the photo, NASA added.

Original Summary: NAS


Review: Urban free climber Alain Robert, popularly known as the 'French Spiderman', has been arrested in the Philippines after climbing an over 700-foot-tall skyscraper in Manila. "I see thousands of people inside the building. I did brighten their day," Robert said when asked about his climb. He holds the Guinness World Record for 'most buildings climbed', having scaled over 120 structures.

Original Summary: 'French Spiderman' arrested for climbing Philippines skyscraper

Predicted Summary: Alain Robert, popularly known as the 'French Spiderman', has been arrested in the Philippines. He holds the Guinness World Record for 'most buildings climbed'

Review: The National Company Law Tribunal has rejected the settlement offer made by Essar Steel's promoters, the Ruia family, to regain control of the company. Ruias made a Ã¢ÂÂ¹54,389-crore offer on the day the Committee of Creditors approved ArcelorMittal's Ã¢ÂÂ¹42,000-crore resolution plan. ArcelorMittal welcomed the ruling and said i


Review: Ahmedabad-headquartered Lambda Therapeutic Research on Tuesday announced the acquisition of US-based Novum Pharmaceutical Research Services. The Indian contract research organisation said this is its second acquisition in North America following the acquisition of the R&D facilities of Canada's Biovail in 2010. Apart from India, Lambda has operations in Canada, Poland and the UK.

Original Summary: India's Lambda buys US research firm Novum Pharmaceutical

Predicted Summary: Lambda Therapeutic Research announced the acquisition of US-based Novum Pharmaceutical Research Services. The Indian contract research organisation said this is its second acquisition in North America. Apart from India, Lambda has

Review: The Income Tax Department on Tuesday said it has confiscated assets worth Ã¢ÂÂ¹6,900 crore so far under the anti-benami transactions law. Those who "abet and induce" benami transactions, benamidar and beneficiaries may face rigorous imprisonment up to 7 years besides be


Review: Actor Rajkummar Rao has said that he finds it "overwhelming" to be compared to Ranbir Kapoor and Ranveer Singh. "I take it as a compliment...I take it with a lot of gratitude because you are comparing me with two great actors,Ã¢ÂÂ he added. Rajkummar further said he doesn't take competition into account and only hopes to grow as an actor. 

Original Summary: Overwhelming to be compared to Ranbir, Ranveer: Rajkummar

Predicted Summary: Rajkummar Rao finds it "overwhelming" to be compared to Ranbir Kapoor and Ranveer Singh. He said he doesn't take competition into account and only hopes to grow as

Review: Shah Rukh Khan was dismissed of the charge of being a beneficiary of benami property in Maharashtra's Alibaug. The Income Tax Department issued an attachment order in February 2018, stating that the property was held by the company 'Deja Vu Farms', further naming Shah Rukh as a beneficiary. The Adjudicating Authority (AA) revoked the order, calling it 'baseless' and 'coloure

In [7]:
average_f1_rouge1 = sum(f1_scores_rouge1) / len(f1_scores_rouge1)
average_f1_rouge2 = sum(f1_scores_rouge2) / len(f1_scores_rouge2)
average_f1_rougeL = sum(f1_scores_rougeL) / len(f1_scores_rougeL)

print("Average F1 Score for ROUGE-1:", average_f1_rouge1)
print("Average F1 Score for ROUGE-2:", average_f1_rouge2)
print("Average F1 Score for ROUGE-L:", average_f1_rougeL)

Average F1 Score for ROUGE-1: 0.3130034028688957
Average F1 Score for ROUGE-2: 0.12115269642142497
Average F1 Score for ROUGE-L: 0.26217923070635807
